In [6]:
import os
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter

In [7]:
def load_data(filepath):
    docs=SimpleDirectoryReader(filepath).load_data()
    parser=SentenceSplitter(chunk_size=500,chunk_overlap=20)
    chunks=parser.get_nodes_from_documents(docs)
    return chunks

In [8]:
%cd ..

d:\1 Kathir-Hope AI\Projects\GenAI\Clinical_Assistant


In [9]:
chunks=load_data('data')

In [10]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import StorageContext,Settings
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

In [ ]:
def vector_store(chunks):
    embed_model=OpenAIEmbedding(api_key="")
    Settings.embed_model=embed_model
    
    db=chromadb.PersistentClient(path="./db")
    chroma_collection=db.get_or_create_collection("clinical_data")
    
    vectorstore=ChromaVectorStore(chroma_collection=chroma_collection)
    
    Storage=StorageContext.from_defaults(vector_store=vectorstore)
    index=VectorStoreIndex.from_documents(documents=chunks,storage_context=Storage)
    return index

In [12]:
index=vector_store(chunks)

In [13]:
# embed_model=HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Settings.embed_model=embed_model
# db=chromadb.PersistentClient(path="./chroma_db")
# chroma_collection=db.get_or_create_collection("clinical_data")
# vector_store=ChromaVectorStore(chroma_collection=chroma_collection)
# Storage=StorageContext.from_defaults(vector_store=vector_store)
# index=VectorStoreIndex.from_vector_store(vector_store)

In [ ]:
from llama_index.llms.openai import OpenAI
import torch
import torch.cuda

llm_model=OpenAI(model_name="gpt-3.5-turbo",api_key="",
                 temperature=0,
                 system_prompt="You are a helpful and professional clinical assistant. Your sole purpose is to provide accurate answers to health and medical questions. Use ONLY the information provided in the given context. If the context does not contain the information needed to answer the question, politely state that you cannot answer based on the provided data."
)
Settings.llm=llm_model

In [23]:
query_engine = index.as_query_engine(similarity_top_k=3)

print("Ask a clinical question or type 'exit' to quit.")
while True:
    query = input("User: ")
    if query.lower() == "exit":
        break
    
    response = query_engine.query(query)
    print(f"Bot: {response}")

Ask a clinical question or type 'exit' to quit.
Bot: Fever is a symptom that should prompt notification of a doctor when it develops during a flare-up of a condition like atopic dermatitis.
Bot: I cannot answer that question based on the provided data.
Bot: I cannot provide an answer to the query about "LLM" based on the provided data.
Bot: ACE inhibitors are commonly used to treat high blood pressure. It is important for individuals with high blood pressure to continue seeing their physicians regularly, take their medication as prescribed, and follow any dietary or lifestyle recommendations provided by their healthcare professional. It is crucial not to stop taking the medication without consulting the prescribing physician. Additionally, individuals should be cautious about taking other medications alongside ACE inhibitors without first consulting their physician, as some medications may interact and affect blood pressure.
